##Portfolio Peformance Table##

**Columns Kept From Assette:**

PORTFOLIOCODE: Primary key to identify the portfolio

DATE: Snapshot date of performance

CURRENCY: Reporting currency for the portfolio (usually 'USD')

BENCHMARKCODE: Identifier for the benchmark (e.g., 'SP500ESG')

**Columns Created from Yfinance Faker:**

ACCOUNTLEVELRETURN: Daily return of the portfolio, computed using actual security prices

YTDRETURN: Return from start of the year to the current date

BENCHMARKRETURN: Benchmarks return over the same daily period
ALPHA	Excess return of portfolio over benchmark (daily)

CALCULATED_AT: Timestamp when the performance was calculated

TICKERS	Comma-separated list of securities in the portfolio

WEIGHTS	Comma-separated list of portfolio weights for the securities


In [ ]:
!pip install yfinance faker

In [ ]:
import yfinance as yf
import pandas as pd
import random
from datetime import datetime

NUM_PORTFOLIOS = 5
SECURITIES_PER_PORTFOLIO = 5
TODAY = datetime.today().strftime('%Y-%m-%d')
YEAR_START = f"{datetime.today().year}-01-01"

esg_tickers = ['AAPL', 'MSFT', 'ADBE', 'NVDA', 'TSLA', 'GOOGL', 'AMZN', 'INTC', 'META', 'V']
rows = []

In [ ]:
# Benchmark setup
benchmark_ticker = "^SPESG"
benchmark_data = yf.download(benchmark_ticker, period="2d", interval="1d", progress=False)
if 'Adj Close' in benchmark_data.columns:
    benchmark_prices = benchmark_data['Adj Close']
elif 'Close' in benchmark_data.columns:
    benchmark_prices = benchmark_data['Close']
else:
    benchmark_prices = None

try:
    benchmark_return = round(((benchmark_prices.iloc[-1] - benchmark_prices.iloc[-2]) / benchmark_prices.iloc[-2]), 4)
except:
    benchmark_return = 0.0

/tmp/ipython-input-2897469290.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  benchmark_data = yf.download(benchmark_ticker, period="2d", interval="1d", progress=False)


In [ ]:
# Portfolio generation
portfolio_data = []
for i in range(NUM_PORTFOLIOS):
    portfolio_code = f"PF{str(i+1).zfill(3)}"
    performance_account = f"PA-{random.randint(1000, 9999)}"
    product_code = f"PRD{str(i+1).zfill(4)}"
    selected_ticker = random.sample(esg_tickers, SECURITIES_PER_PORTFOLIO)
    weights = [random.uniform(0.1, 0.4) for _ in range(SECURITIES_PER_PORTFOLIO)]
    total = sum(weights)
    normalized_weights = [round(w / total, 4) for w in weights]

    portfolio_data.append({
      "PORTFOLIOCODE": portfolio_code,
      "PERFORMANCEACCOUNT": performance_account,
      "PRODUCTCODE": product_code,
      "TICKERS": selected_ticker,
      "WEIGHTS": normalized_weights
})

In [ ]:
# Calculation
for p in portfolio_data:
    tickers = p["TICKERS"]
    weights = p["WEIGHTS"]
    performance_account = p["PERFORMANCEACCOUNT"]
    portfolio_code = p["PORTFOLIOCODE"]
    product_code = p["PRODUCTCODE"]

    try:
        price_data_full = yf.download(tickers, period="2d", interval="1d", progress=False)
        if 'Adj Close' in price_data_full.columns:
            price_data = price_data_full[['Adj Close']]
        elif 'Close' in price_data_full.columns:
            price_data = price_data_full[['Close']]
        else:
            continue
    except Exception as e:
        print(f"⚠️ Skipping {portfolio_code} due to error: {e}")
        continue

    try:
        prev_close = price_data.iloc[-2].values[0]
        curr_price = price_data.iloc[-1].values[0]
        account_return = (curr_price - prev_close) / prev_close
    except:
        continue

    try:
        ytd_data_full = yf.download(tickers, start=YEAR_START, end=TODAY, progress=False)
        if 'Adj Close' in ytd_data_full.columns:
            ytd_data = ytd_data_full[['Adj Close']]
        elif 'Close' in ytd_data_full.columns:
            ytd_data = ytd_data_full[['Close']]
        else:
            continue

        ytd_start = ytd_data.iloc[0].values[0]
        ytd_current = ytd_data.iloc[-1].values[0]
        ytd_portfolio_return = (ytd_current - ytd_start) / ytd_start
    except:
        continue

    alpha = round(account_return - benchmark_return, 4)

    rows.append({
        "PORTFOLIOCODE": portfolio_code,
        "PERFORMANCEACCOUNT": performance_account,
        "PRODUCTCODE": product_code,
        "DATE": TODAY,
        "ACCOUNTLEVELRETURN": round(account_return, 4),
        "YTDRETURN": round(ytd_portfolio_return, 4),
        "BENCHMARKRETURN": benchmark_return,
        "ALPHA": alpha,
        "BENCHMARKCODE": "SP500ESG",
        "CURRENCY": "USD",
        "CALCULATED_AT": datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'),
        "TICKERS": ", ".join(tickers),
        "WEIGHTS": ", ".join([str(w) for w in weights])
    })

/tmp/ipython-input-930851628.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data_full = yf.download(tickers, period="2d", interval="1d", progress=False)
/tmp/ipython-input-930851628.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ytd_data_full = yf.download(tickers, start=YEAR_START, end=TODAY, progress=False)
/tmp/ipython-input-930851628.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data_full = yf.download(tickers, period="2d", interval="1d", progress=False)
/tmp/ipython-input-930851628.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ytd_data_full = yf.download(tickers, start=YEAR_START, end=TODAY, progress=False)
/tmp/ipython-input-930851628.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data_full = yf.download(tickers, period="2d", interval="1d", progress=False)
/tmp/ip

In [ ]:
# Output
df = pd.DataFrame(rows)
timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
df.to_csv(f"portfolio_performance_{timestamp}.csv", index=False)
df

,PORTFOLIOCODE,PERFORMANCEACCOUNT,PRODUCTCODE,DATE,ACCOUNTLEVELRETURN,YTDRETURN,BENCHMARKRETURN,ALPHA,BENCHMARKCODE,CURRENCY,CALCULATED_AT,TICKERS,WEIGHTS
0,PF001,PA-7304,PRD0001,2025-08-05,0.0048,-0.1641,Ticker ^SPESG 0.0168 dtype: float64,Ticker ^SPESG -0.012 dtype: float64,SP500ESG,USD,2025-08-05 03:14:35,"AMZN, INTC, TSLA, META, AAPL","0.2083, 0.092, 0.2503, 0.2228, 0.2265"
1,PF002,PA-6573,PRD0002,2025-08-05,-0.0144,-0.0389,Ticker ^SPESG 0.0168 dtype: float64,Ticker ^SPESG -0.0312 dtype: float64,SP500ESG,USD,2025-08-05 03:14:37,"MSFT, TSLA, AMZN, V, NVDA","0.0695, 0.2178, 0.2173, 0.2694, 0.226"
2,PF003,PA-9317,PRD0003,2025-08-05,-0.0257,-0.2316,Ticker ^SPESG 0.0168 dtype: float64,Ticker ^SPESG -0.0425 dtype: float64,SP500ESG,USD,2025-08-05 03:14:38,"MSFT, TSLA, NVDA, V, ADBE","0.2562, 0.204, 0.2599, 0.2019, 0.078"
3,PF004,PA-6310,PRD0004,2025-08-05,0.0048,-0.1641,Ticker ^SPESG 0.0168 dtype: float64,Ticker ^SPESG -0.012 dtype: float64,SP500ESG,USD,2025-08-05 03:14:40,"INTC, AAPL, MSFT, ADBE, AMZN","0.1625, 0.1629, 0.2555, 0.1169, 0.3022"
4,PF005,PA-2133,PRD0005,2025-08-05,-0.0257,-0.2316,Ticker ^SPESG 0.0168 dtype: float64,Ticker ^SPESG -0.0425 dtype: float64,SP500ESG,USD,2025-08-05 03:14:42,"META, ADBE, V, GOOGL, NVDA","0.2625, 0.1022, 0.1765, 0.1927, 0.2661"
